## Setup the kaggle environment

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!git clone https://github.com/benihime91/leaf-disease-classification-kaggle.git
!pip install hydra-core timm --upgrade --quiet

import sys
sys.path.append("/kaggle/working/leaf-disease-classification-kaggle/")

Cloning into 'leaf-disease-classification-kaggle'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 289 (delta 142), reused 196 (delta 66), pack-reused 0
Receiving objects: 100% (289/289), 8.38 MiB | 10.66 MiB/s, done.
Resolving deltas: 100% (142/142), done.


## Import depedencies

In [3]:
import os
from experiment import run
from hydra.experimental import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import time

## Setting up the config pipeline

In [4]:
# setting up config paths
config_dir = "/kaggle/working/leaf-disease-classification-kaggle/conf/"
config_name = "config"
model_arch = "efficientnetb0"

# which fold to train on ?
fold_num = 0 

# setting up the configuration
# see: https://hydra.cc/docs/next/advanced/override_grammar/basic/
image_dir = "/kaggle/input/cassava-leaf-disease-classification/train_images/"
csv_dir   = "/kaggle/input/cassava-leaf-disease-classification/train.csv"
json_dir  = "/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
fold_csv_dir = "/kaggle/working/leaf-disease-classification-kaggle/data/fold_df.csv"

# setting up model save paths and job_name
run_name = f"{model_arch}_fold_{fold_num}_{int(time.time())}"
checkpoint_path = "/kaggle/working/"
model_save_dir = f"/kaggle/working/{model_arch}_fold_{fold_num}.pt"

# configuring the training job
batch_size = 128
num_epochs = 20
steps_per_epoch = 17118 // batch_size
num_classes = 5

optimizer = "adamw"
scheduler = "onecyclelr"
learning_rate = 0.002
weight_decay = 0.001
gradient_clip = 0.1

# custom parameters to override the default configs
customs = [
    f"model={model_arch}",
    f"optimizer={optimizer}", 
    f"scheduler={scheduler}",
    f"image_dir={image_dir}",
    f"csv_dir={csv_dir}",
    f"json_dir={json_dir}",
    f"fold_csv_dir={fold_csv_dir}",
    f"fold_num={fold_num}",
    f"run_name={run_name}",
    f"checkpoint_path={checkpoint_path}",
    f"model_save_dir={model_save_dir}",
]

## Initializing Hydra Config

In [5]:
# from : https://github.com/facebookresearch/hydra/blob/master/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
with initialize_config_dir(config_dir=config_dir):
    # override defaults with customs list
    cfg = compose(config_name=config_name, overrides=customs)
    
    # overide with custom training configuration
    cfg.training.num_epochs  = num_epochs
    cfg.training.num_classes = num_classes
    cfg.training.lr = learning_rate
    cfg.training.steps_per_epoch = steps_per_epoch
    cfg.optimizer.params.weight_decay = weight_decay
    cfg.lightning.init_args.gradient_clip_val = gradient_clip
    cfg.model.use_custom_base = True
    
    # display configuration
    print(OmegaConf.to_yaml(cfg))

image_dir: /kaggle/input/cassava-leaf-disease-classification/train_images/
csv_dir: /kaggle/input/cassava-leaf-disease-classification/train.csv
json_dir: /kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json
fold_csv_dir: /kaggle/working/leaf-disease-classification-kaggle/data/fold_df.csv
use_weights: true
fold_num: 0
run_name: efficientnetb0_fold_0_1606139094
checkpoint_path: /kaggle/working/
model_save_dir: /kaggle/working/efficientnetb0_fold_0.pt
seed: 42
model:
  class_name: timm.create_model
  params:
    pretrained: true
    model_name: tf_efficientnet_b0
  output_dims: 1000
  fc1: 512
  fc2: 256
  num_classes: ${training.num_classes}
  use_custom_base: true
training:
  seed: ${seed}
  job_name: ${run_name}
  checkpoint_path: ${checkpoint_path}
  model_save_dir: ${model_save_dir}
  num_epochs: 20
  steps_per_epoch: 133
  lr: 0.002
  num_classes: 5
  image_dim: 224
  metric: val_loss
  dataloaders:
    batch_size: 128
    pin_memory: true
    num_workers:

## Run Training Pipeline

In [ ]:
# train model
run(cfg, print_layers=True)